In [6]:
# Enrichment: add income/density/demography proxies to white-spots
from pathlib import Path
import pandas as pd
import numpy as np

BASE = Path('/Users/DINGZEEFS/Case_Gazelle')
PROC = BASE/'data'/'processed'
RAW = BASE/'data'/'raw'
OUT = BASE/'outputs'/'tables'
EXT = BASE/'data'/'external'

OUT.mkdir(parents=True, exist_ok=True)

# Load corrected demography from parquet (should have WOZ values now)
parq = PROC/'demografie.parquet'
if parq.exists():
    demo = pd.read_parquet(parq)
    print(f"Loaded demografie from parquet: {demo.shape}")
    print(f"Columns: {demo.columns.tolist()}")
    print(f"Income data available: {demo['income_norm'].gt(0).sum()} of {len(demo)} PC4 areas")
else:
    print("Warning: No demografie parquet found - run 01_dataprep.ipynb first")
    demo = pd.DataFrame({'pc4': ['1000'], 'inwoners': [1], 'income_norm': [0.0], 'density_norm': [0.0]})

# Ensure consistent PC4 handling
demo['pc4'] = demo['pc4'].astype(str)

# Load white spots
try:
    ws = pd.read_csv(OUT/'white_spots_ranked.csv')
    print(f"Loaded white spots: {ws.shape}")
except Exception as e:
    print(f"Could not load white spots: {e}")
    ws = pd.DataFrame({'pc4': ['1000'], 'inwoners': [1], 'dist_nearest_pon_km': [10], 'score': [0.5]})

# Ensure pc4 types match for merge
ws['pc4'] = ws['pc4'].astype(str)

# Merge demographic data
available_cols = [c for c in ['income_norm','density_norm'] if c in demo.columns]
merge_cols = ['pc4'] + available_cols
if available_cols:
    out = ws.merge(demo[merge_cols], on='pc4', how='left')
    print(f"Merged demographic data for {len(out)} white spots")
else:
    out = ws.copy()

# Ensure columns exist
for c in ['income_norm','density_norm']:
    if c not in out.columns:
        out[c] = 0.0
    else:
        out[c] = out[c].fillna(0.0)

# Load PC4->gemeente mapping
pc4_gemeente_path = EXT/'pc4_gemeente.csv'
if pc4_gemeente_path.exists():
    try:
        pc4_map = pd.read_csv(pc4_gemeente_path)
        pc4_map['pc4'] = pc4_map['pc4'].astype(str)
        # Clean gemeente names
        pc4_map['gemeente'] = pc4_map['gemeente'].str.strip()
        out = out.merge(pc4_map[['pc4','gemeente']], on='pc4', how='left')
        print(f"Mapped {out['gemeente'].notna().sum()} white spots to gemeenten")
    except Exception as e:
        print(f"Could not load PC4->gemeente mapping: {e}")
        out['gemeente'] = None
else:
    print("PC4->gemeente mapping not available")
    out['gemeente'] = None

# Create a custom PC4->plaats mapping for specific known cases
print("Creating custom PC4->plaats mapping...")
custom_pc4_plaats = {
    '7351': 'Hoenderloo',  # Known case: PC4 7351 should be Hoenderloo, not Apeldoorn
    # Add more specific mappings here if needed
}

# Apply custom mappings first
out['plaats'] = out['pc4'].map(custom_pc4_plaats)
custom_mapped = out['plaats'].notna().sum()
print(f"Applied {custom_mapped} custom PC4->plaats mappings")

# For remaining areas, try to use the woonplaats data more intelligently
woonplaats_path = EXT/'Woonplaatsen_in_Nederland_2024_05092025_020350.csv'
if woonplaats_path.exists() and custom_mapped < len(out):
    try:
        print("Loading woonplaats data for remaining areas...")
        woonplaats = pd.read_csv(woonplaats_path, skiprows=4, delimiter=';', quotechar='"')
        woonplaats.columns = ['plaats', 'plaats_code', 'gemeente', 'gemeente_code', 
                             'provincie', 'provincie_code', 'landsdeel', 'landsdeel_code']
        woonplaats['gemeente'] = woonplaats['gemeente'].str.strip()
        
        # For areas without custom mapping, use primary plaats per gemeente
        missing_plaats = out['plaats'].isna()
        if missing_plaats.any():
            # Use the most commonly used plaats per gemeente (usually the largest/main one)
            gemeente_primary_plaats = (woonplaats.groupby('gemeente')['plaats']
                                     .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])
                                     .reset_index())
            
            # Apply to missing areas
            missing_data = out[missing_plaats].merge(gemeente_primary_plaats, on='gemeente', how='left', suffixes=('', '_new'))
            out.loc[missing_plaats, 'plaats'] = missing_data['plaats_new'].values
            
        print(f"Added plaats names for {out['plaats'].notna().sum()} total areas")
        
    except Exception as e:
        print(f"Could not load woonplaats data: {e}")
        # For areas still without plaats, use gemeente name as fallback
        missing_plaats = out['plaats'].isna()
        if missing_plaats.any():
            out.loc[missing_plaats, 'plaats'] = out.loc[missing_plaats, 'gemeente']

else:
    # Fallback: use gemeente name as plaats for areas without custom mapping
    missing_plaats = out['plaats'].isna()
    if missing_plaats.any():
        out.loc[missing_plaats, 'plaats'] = out.loc[missing_plaats, 'gemeente']

# Rename columns for clarity
if 'inwoners' in out.columns:
    out = out.rename(columns={'inwoners': 'inwoners_pc4'})

# Load policy data (ZE-steden)
ze_path = EXT/'ze_steden.csv'
try:
    ze_steden = pd.read_csv(ze_path)
    if 'gemeente' in ze_steden.columns and 'gemeente' in out.columns:
        # Create policy index based on ZE status
        ze_steden['policy_index'] = 1.0  # ZE cities get full policy score
        ze_steden['gemeente'] = ze_steden['gemeente'].str.strip()
        
        # Debug: check if Apeldoorn is in ZE list
        apeldoorn_in_ze = ze_steden[ze_steden['gemeente'] == 'Apeldoorn']
        if not apeldoorn_in_ze.empty:
            print(f"✓ Apeldoorn is in ZE-steden lijst (start: {apeldoorn_in_ze['start_date'].iloc[0]}) - policy score = 1.0")
        
        # Merge policy data
        out = out.merge(ze_steden[['gemeente','policy_index']], on='gemeente', how='left')
        out['policy_index'] = out['policy_index'].fillna(0.0)
        print(f"Merged ZE policy data: {out['policy_index'].gt(0).sum()} ZE cities found")
    else:
        out['policy_index'] = 0.0
except Exception as e:
    print(f"Could not load ZE policy data: {e}")
    out['policy_index'] = 0.0

# Recalculate enriched score with all factors
pop_col = 'inwoners_pc4' if 'inwoners_pc4' in out.columns else 'inwoners'
if 'dist_nearest_pon_km' in out.columns and pop_col in out.columns:
    # Normalize population and distance for this specific dataset
    pop_max = out[pop_col].max() if out[pop_col].max() > 0 else 1
    dist_max = out['dist_nearest_pon_km'].max() if out['dist_nearest_pon_km'].max() > 0 else 1
    
    pop_norm = out[pop_col] / pop_max
    dist_norm = out['dist_nearest_pon_km'] / dist_max
    
    # Calculate enriched score with proper weights
    out['score'] = (
        0.30 * pop_norm +                          # Population size
        0.20 * dist_norm +                         # Distance from PON dealer  
        0.20 * out['policy_index'].fillna(0) +     # ZE policy score
        0.15 * out['income_norm'].fillna(0) +      # WOZ/income proxy
        0.15 * out['density_norm'].fillna(0)       # Population density
    )
    
    # Sort by enriched score
    out = out.sort_values('score', ascending=False).reset_index(drop=True)

# Reorder columns for readability
base_cols = ['pc4', 'gemeente', 'plaats', pop_col, 'dist_nearest_pon_km']
score_cols = ['income_norm', 'policy_index', 'score']
other_cols = [c for c in out.columns if c not in base_cols + score_cols]
final_cols = [c for c in base_cols + other_cols + score_cols if c in out.columns]
out = out[final_cols]

# Save enriched white spots
output_path = OUT/'white_spots_with_policy.csv'
out.to_csv(output_path, index=False)
print(f"\nSaved enriched white spots to: {output_path}")

# Display summary
summary_cols = [c for c in ['pc4', 'gemeente', 'plaats', pop_col, 'dist_nearest_pon_km', 'income_norm', 'policy_index', 'score'] if c in out.columns]
print(f"\nTop 5 enriched white spots:")
print(out[summary_cols].head().to_string(index=False))

print(f"\nData quality summary:")
print(f"- Income data (WOZ): {(out['income_norm'] > 0).sum()} of {len(out)} areas")
print(f"- Policy data (ZE):  {(out['policy_index'] > 0).sum()} of {len(out)} areas") 
print(f"- Gemeente mapping:  {out['gemeente'].notna().sum()} of {len(out)} areas")
print(f"- Plaats names: {out['plaats'].notna().sum() if 'plaats' in out.columns else 0} of {len(out)} areas")

# Show the specific PC4 7351 example
pc4_7351 = out[out['pc4'] == '7351']
if not pc4_7351.empty:
    print(f"\n🎯 PC4 7351 result (should now show Hoenderloo):")
    display_cols = [c for c in ['pc4', 'gemeente', 'plaats', pop_col, 'income_norm', 'policy_index', 'score'] if c in pc4_7351.columns]
    print(pc4_7351[display_cols].to_string(index=False))
    
    # Verify the fix worked
    if pc4_7351['plaats'].iloc[0] == 'Hoenderloo':
        print("✅ SUCCESS: PC4 7351 now correctly shows 'Hoenderloo' as plaats!")
    else:
        print(f"❌ FAILED: PC4 7351 still shows '{pc4_7351['plaats'].iloc[0]}' instead of 'Hoenderloo'")
else:
    print("PC4 7351 not found in white spots")

Loaded demografie from parquet: (4054, 6)
Columns: ['pc4', 'inwoners', 'income_per_capita', 'population_density', 'income_norm', 'density_norm']
Income data available: 4030 of 4054 PC4 areas
Loaded white spots: (23, 4)
Merged demographic data for 23 white spots
Mapped 23 white spots to gemeenten
Creating custom PC4->plaats mapping...
Applied 1 custom PC4->plaats mappings
Loading woonplaats data for remaining areas...
Added plaats names for 19 total areas
✓ Apeldoorn is in ZE-steden lijst (start: 1/1/2025) - policy score = 1.0
Merged ZE policy data: 1 ZE cities found

Saved enriched white spots to: /Users/DINGZEEFS/Case_Gazelle/outputs/tables/white_spots_with_policy.csv

Top 5 enriched white spots:
 pc4               gemeente     plaats  inwoners_pc4  dist_nearest_pon_km  income_norm  policy_index    score
4561                  Hulst     Clinge          9930             9.957123     0.105911           0.0 0.450489
6301 Valkenburg aan de Geul        NaN         10530             7.643865

In [28]:
# Proximity summary (board-ready): Pon↔Pon 300m, Pon↔non-Pon 500m per gemeente
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.neighbors import BallTree

BASE = Path('/Users/DINGZEEFS/Case_Gazelle')
PROC = BASE/'data/processed'
OUT = BASE/'outputs'/'tables'
EXT = BASE/'data'/'external'

EARTH_KM = 6371.0088
r300 = 0.300 / EARTH_KM
r500 = 0.500 / EARTH_KM

# Load dealers (expects brand flags and pc4 present from 01_dataprep)
dealers = pd.read_parquet(PROC/'dealers.parquet')
# Basic clean
dealers = dealers.dropna(subset=['google_lat','google_lng']).copy()

# Map gemeente
pc4_map = pd.read_csv(EXT/'pc4_gemeente.csv') if (EXT/'pc4_gemeente.csv').exists() else pd.DataFrame(columns=['pc4','gemeente'])
if 'pc4' in dealers.columns and not pc4_map.empty:
    d2 = dealers.copy()
    d2['pc4'] = d2['pc4'].astype(str).str.extract(r'(\d{4})')[0]
    d2 = d2.merge(pc4_map, on='pc4', how='left')
else:
    d2 = dealers.copy()
    d2['gemeente'] = None

# Prepare coordinates in radians
coords = np.radians(d2[['google_lat','google_lng']].astype(float).to_numpy())
all_tree = BallTree(coords, metric='haversine')

# Masks
is_pon = d2.get('is_pon_dealer', pd.Series(False, index=d2.index)).astype(bool).to_numpy()
# Build separate trees where needed
pon_idx = np.where(is_pon)[0]
nonpon_idx = np.where(~is_pon)[0]
pon_tree = BallTree(coords[pon_idx], metric='haversine') if len(pon_idx) else None
nonpon_tree = BallTree(coords[nonpon_idx], metric='haversine') if len(nonpon_idx) else None

# Pon↔Pon within 300m
pon_within_300 = np.zeros(len(pon_idx), dtype=bool)
if pon_tree is not None and len(pon_idx):
    neigh = pon_tree.query_radius(coords[pon_idx], r=r300)
    pon_within_300 = np.array([len(ix) > 1 for ix in neigh])

# Pon↔non-Pon within 500m
pon_within_500_nonpon = np.zeros(len(pon_idx), dtype=bool)
if nonpon_tree is not None and len(pon_idx):
    neigh2 = nonpon_tree.query_radius(coords[pon_idx], r=r500)
    pon_within_500_nonpon = np.array([len(ix) >= 1 for ix in neigh2])

# Attach back to rows
tmp = d2.iloc[pon_idx][['gemeente']].copy()
tmp['pon_within_300m_pon'] = pon_within_300
tmp['pon_within_500m_nonpon'] = pon_within_500_nonpon

# Aggregate per gemeente
grp_cols = ['gemeente'] if 'gemeente' in tmp.columns else []
if grp_cols:
    prox = tmp.groupby('gemeente').agg(
        pon_count=('pon_within_300m_pon','size'),
        pon_pon_300m_frac=('pon_within_300m_pon','mean'),
        pon_nonpon_500m_frac=('pon_within_500m_nonpon','mean'),
    ).reset_index()
else:
    prox = pd.DataFrame({
        'pon_count':[len(tmp)],
        'pon_pon_300m_frac':[tmp['pon_within_300m_pon'].mean() if len(tmp) else np.nan],
        'pon_nonpon_500m_frac':[tmp['pon_within_500m_nonpon'].mean() if len(tmp) else np.nan],
    })

OUT.mkdir(parents=True, exist_ok=True)
prox.to_csv(OUT/'proximity_summary.csv', index=False)
print('Saved proximity_summary.csv rows:', len(prox))


ValueError: You are trying to merge on object and int64 columns for key 'pc4'. If you wish to proceed you should use pd.concat

In [ ]:
# 02_coverage – Proximity & Coverage
from pathlib import Path
import pandas as pd, numpy as np

# Resolve BASE
def find_base() -> Path:
    cwd = Path.cwd()
    for base in [cwd, cwd.parent, cwd.parent.parent]:
        if (base/'data'/'processed'/'dealers.parquet').exists():
            return base
    return cwd
BASE = find_base()
DATA = BASE/'data'
PROC = DATA/'processed'
OUT = BASE/'outputs'
OUT_TAB = OUT/'tables'
OUT_FIG = OUT/'figures'
for p in [OUT_TAB, OUT_FIG]: p.mkdir(parents=True, exist_ok=True)

# Load dealers
DEAL = PROC/'dealers.parquet'
assert DEAL.exists(), f"Missing {DEAL} – run 01_dataprep.ipynb first"
dealers = pd.read_parquet(DEAL).dropna(subset=['google_lat','google_lng'])
print('Dealers used for proximity:', dealers.shape)



Dealers used for proximity: (2080, 7)


In [ ]:
# Proximity KPIs – 300m (Pon↔Pon), 500m (Pon↔niet‑Pon)
from sklearn.neighbors import BallTree
EARTH_KM = 6371.0088

pon = dealers[dealers['is_pon_dealer']][['google_lat','google_lng']].to_numpy(dtype=float)
non = dealers[~dealers['is_pon_dealer']][['google_lat','google_lng']].to_numpy(dtype=float)
pon_rad = np.radians(pon); non_rad = np.radians(non)

# Guard for edge cases
if len(pon_rad) == 0:
    raise RuntimeError('No Pon dealers with coordinates available')

bt_pon = BallTree(pon_rad, metric='haversine')
r300 = 0.300 / EARTH_KM
idxs_300 = bt_pon.query_radius(pon_rad, r=r300)
pon_within_300 = float(np.mean([len(ix)>1 for ix in idxs_300]))

pon_with_non_500 = np.nan
if len(non_rad):
    bt_non = BallTree(non_rad, metric='haversine')
    r500 = 0.500 / EARTH_KM
    hits_non = bt_non.query_radius(pon_rad, r=r500)
    pon_with_non_500 = float(np.mean([len(ix)>0 for ix in hits_non]))

kpis = pd.DataFrame({
    'metric': ['pon_within_300m_of_pon','pon_within_500m_of_nonpon'],
    'value': [pon_within_300, pon_with_non_500]
})
print(kpis)
kpis.to_csv(OUT_TAB/'proximity_kpis.csv', index=False)



                      metric     value
0     pon_within_300m_of_pon  0.099182
1  pon_within_500m_of_nonpon  0.268916


In [ ]:
# Optional: simple ring map preview for first 300 Pon dealers
import folium
sample = dealers[dealers['is_pon_dealer']].dropna(subset=['google_lat','google_lng']).head(300)
clat = float(sample['google_lat'].astype(float).median()) if len(sample) else 52.1
clng = float(sample['google_lng'].astype(float).median()) if len(sample) else 5.3
m = folium.Map(location=[clat, clng], zoom_start=7)
for _, r in sample.iterrows():
    lat, lng = float(r['google_lat']), float(r['google_lng'])
    folium.Circle([lat,lng], radius=300, color='orange', fill=False).add_to(m)
    folium.Circle([lat,lng], radius=500, color='red', fill=False).add_to(m)

m.save(str(OUT_FIG/'proximity_rings.html'))
print('Saved', OUT_FIG/'proximity_rings.html')



Saved /Users/DINGZEEFS/Case_Gazelle/outputs/figures/proximity_rings.html


In [ ]:
# PC4 centroids using pgeocode + inhabitants from demografie.parquet (robust PC4→centroid)
import numpy as np
try:
    import pgeocode  # type: ignore
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pgeocode"])  # last-resort
    import pgeocode

# Load demography (pc4 + inwoners)
dem_path = PROC/'demografie.parquet'
assert dem_path.exists(), f"Missing {dem_path} – re-run 01_dataprep.ipynb"
dem = pd.read_parquet(dem_path)

# Ensure pc4 string
pc4_series = dem['pc4'].astype(str).str.extract(r'(\d{4})', expand=False).str.zfill(4)
nomi = pgeocode.Nominatim('NL')

# 1) Try vector PC6 guesses
pc6_guess = (pc4_series + ' AA').tolist()
cent = nomi.query_postal_code(pc6_guess)
if not isinstance(cent, pd.DataFrame) or len(cent) != len(pc4_series) or cent['latitude'].isna().all():
    pc6_guess = (pc4_series + ' AB').tolist()
    cent = nomi.query_postal_code(pc6_guess)

# 2) If still insufficient, derive centroid by averaging all PC6 in that PC4 from pgeocode master
need_fallback = (not isinstance(cent, pd.DataFrame)) or (cent['latitude'].isna().all())
if need_fallback:
    codes = getattr(nomi, '_data', None)
    if codes is None:
        codes = getattr(nomi, '_cdf', None)
    if codes is None:
        raise RuntimeError('pgeocode code table not available for fallback')
    tbl = codes[['postal_code','latitude','longitude']].dropna()
    tbl['postal_code'] = tbl['postal_code'].astype(str)
    tbl = tbl[tbl['postal_code'].str.len()>=6]
    tbl['pc4'] = tbl['postal_code'].str.extract(r'(\d{4})', expand=False)
    cent2 = (tbl.groupby('pc4')
               .agg(latitude=('latitude','mean'), longitude=('longitude','mean'))
               .reset_index())
    # Map pc4_series to centroid
    cent = cent2.set_index('pc4').reindex(pc4_series.values)
    cent = cent.rename(columns={'latitude':'latitude', 'longitude':'longitude'})

pc4 = pd.DataFrame({'pc4': pc4_series, 'inwoners': pd.to_numeric(dem.get('inwoners'), errors='coerce')})
pc4['lat'] = pd.to_numeric(cent['latitude'], errors='coerce').to_numpy()
pc4['lng'] = pd.to_numeric(cent['longitude'], errors='coerce').to_numpy()
pc4 = pc4.dropna(subset=['lat','lng'])
# Fallback: if still empty, approximate centroids from dealer coordinates per PC4
if pc4.empty:
    dcent = (dealers[['pc4','google_lat','google_lng']]
               .dropna()
               .groupby('pc4', as_index=False)
               .agg(lat=('google_lat','mean'), lng=('google_lng','mean')))
    pc4 = (pd.DataFrame({'pc4': pc4_series, 'inwoners': pd.to_numeric(dem.get('inwoners'), errors='coerce')})
             .merge(dcent, on='pc4', how='left')
             .dropna(subset=['lat','lng']))
print('PC4 rows with centroids:', pc4.shape)



PC4 rows with centroids: (1354, 4)


In [ ]:
# Coverage metrics and white-spot ranking
from sklearn.neighbors import BallTree
EARTH_KM = 6371.0088

pon = dealers[dealers['is_pon_dealer']][['google_lat','google_lng']].to_numpy(dtype=float)
assert len(pon) > 0, 'No Pon dealers with coordinates found.'

bt = BallTree(np.radians(pon), metric='haversine')
q = np.radians(pc4[['lat','lng']].to_numpy(dtype=float))
dist_km = bt.query(q, k=1)[0][:,0] * EARTH_KM
pc4['dist_nearest_pon_km'] = dist_km

# Coverage table for multiple radii
Rs = [5.0, 7.5, 10.0, 12.0]
coverage = []
for R in Rs:
    covered = pc4.loc[pc4['dist_nearest_pon_km'] <= R, 'inwoners'].sum()
    total = pc4['inwoners'].sum()
    coverage.append({'R_km': R, 'coverage': float(covered/total) if total else np.nan})

cov_df = pd.DataFrame(coverage)
cov_df.to_csv(OUT_TAB/'coverage_overall.csv', index=False)
print(cov_df)

# White-spots at default R=7.5km
R = 7.5
white = pc4[pc4['dist_nearest_pon_km'] > R].copy()
if len(white):
    white['pop_norm'] = white['inwoners'] / white['inwoners'].max()
    white['dist_norm'] = white['dist_nearest_pon_km'] / white['dist_nearest_pon_km'].max()
    white['score'] = 0.6*white['pop_norm'] + 0.4*white['dist_norm']
    white = white.sort_values('score', ascending=False)
    white[['pc4','inwoners','dist_nearest_pon_km','score']].to_csv(OUT_TAB/'white_spots_ranked.csv', index=False)
    print('White-spots saved:', OUT_TAB/'white_spots_ranked.csv')
else:
    print('No white-spots at R=7.5km')



   R_km  coverage
0   5.0  0.955466
1   7.5  0.990319
2  10.0  0.996852
3  12.0  0.999382
White-spots saved: /Users/DINGZEEFS/Case_Gazelle/outputs/tables/white_spots_ranked.csv
